# Agenda: Mask indexes

- Comparisons
- Broadcasts and comparisons
- Using that to filter our series with a "boolean index" or a "mask index"
- Complex comparisons with "and" and "or"

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
s = Series([10, 20, 30, 40, 50, 60, 70],
           index=list('abcdefg'))

In [3]:
s

a    10
b    20
c    30
d    40
e    50
f    60
g    70
dtype: int64

In [4]:
# I can retrieve any element of the series with either .loc (based on the index) or .iloc (based on the position)

In [5]:
s.loc['d']

40

In [6]:
s.iloc[4]

50

In [7]:
# Inside of the [], I can put a list of locations that I want to retrieve
# this is known as "fancy indexing"

s.loc[['a', 'd']]

a    10
d    40
dtype: int64

In [8]:
s.iloc[[2, 5]]

c    30
f    60
dtype: int64